# Demonstration of McStasScript
This file demonstrates how McStasScript can be used to run McStas from a python environment in a userfreindly manner.

In [1]:
import mcstasscript as ms

# Creating the instance of the class, insert path to mcrun and to mcstas root directory
instr = ms.McStas_instr("jupyter_demo")

The following components are found in the work_directory / input_path:
     Union_sphere.comp
     Texture_process.comp
     Union_cone.comp
     Union_box.comp
     Single_crystal_process.comp
     Union_abs_logger_2D_space.comp
     Union_logger_2D_kf.comp
     Template_process.comp
     PhononSimple_process.comp
     Union_conditional_standard.comp
     Union_abs_logger_1D_space.comp
     Union_abs_logger_event.comp
     NCrystal_process.comp
     Union_abs_logger_1D_space_event.comp
     Union_abs_logger_1D_space_tof.comp
     Union_logger_2D_space.comp
     Union_conditional_PSD.comp
     Union_master.comp
     AF_HB_1D_process.comp
     Union_logger_2D_kf_time.comp
     Union_cylinder.comp
     Union_abs_logger_1D_space_tof_to_lambda.comp
     Powder_process.comp
     Union_make_material.comp
     Incoherent_process.comp
     Union_logger_1D.comp
     Union_logger_3D_space.comp
     IncoherentPhonon_process.comp
     Union_logger_2DQ.comp
     Union_mesh.comp
     Union_logger_2D

The instrument object knows what components are available in this McStas installation and can provide some help.

In [2]:
instr.available_components() # Shows available McStas component categories in current installation

Here are the available component categories:
 contrib
 misc
 monitors
 obsolete
 optics
 samples
 sources
 work directory
Call available_components(category_name) to display


In [3]:
instr.available_components("sources") # Display all McStas source components 

Here are all components in the sources category.
 Adapt_check     Moderator           Source_Optimizer   Source_gen
 ESS_butterfly   Monitor_Optimizer   Source_adapt       Source_simple
 ESS_moderator   Source_Maxwell_3    Source_div         


In [4]:
instr.component_help("Source_simple") # Displays help on the Source_simple component

 ___ Help Source_simple _____________________________________________________________
|optional parameter|required parameter|default value|user specified value|
radius = 0.1 [m] // Radius of circle in (x,y,0) plane where neutrons are 
                    generated. 
yheight = 0.0 [m] // Height of rectangle in (x,y,0) plane where neutrons are 
                     generated. 
xwidth = 0.0 [m] // Width of rectangle in (x,y,0) plane where neutrons are 
                    generated. 
dist = 0.0 [m] // Distance to target along z axis.
focus_xw = 0.045 [m] // Width of target
focus_yh = 0.12 [m] // Height of target
E0 = 0.0 [meV] // Mean energy of neutrons.
dE = 0.0 [meV] // Energy half spread of neutrons (flat or gaussian sigma).
lambda0 = 0.0 [AA] // Mean wavelength of neutrons.
dlambda = 0.0 [AA] // Wavelength half spread of neutrons.
flux = 1.0 [1/(s*cm**2*st*energy unit)] // flux per energy unit, Angs or meV if 
                                           flux=0, the source emits 1 in 4*

First a *Source_simple* component called *Source* is added.

In [5]:
source = instr.add_component("Source", "Source_simple") # Adds an instance of Source_simple

One way to adjust the parameters of a component is to access them directly as attributes.

In [6]:
source.xwidth = 0.06
source.yheight = 0.08;
source.dist = 2
source.focus_xw = 0.05
source.focus_yh = 0.05
source.flux = 1E8

It is possible to create parameter objects and use these directly as input to component parameters.

In [7]:
wavelength = instr.add_parameter("double", "wavelength", value=3, comment="[AA] Wavelength emmited from source")
source.lambda0 = wavelength
source.dlambda = 0.05

In [8]:
print(source) # Verify that the information is correct

COMPONENT Source = Source_simple
  yheight = 0.08 [m]
  xwidth = 0.06 [m]
  dist = 2 [m]
  focus_xw = 0.05 [m]
  focus_yh = 0.05 [m]
  lambda0 = wavelength [AA]
  dlambda = 0.05 [AA]
  flux = 100000000.0 [1/(s*cm**2*st*energy unit)]
AT [0, 0, 0] ABSOLUTE


Next a guide is added and a comment is included.

In [9]:
guide = instr.add_component("Guide", "Guide_gravity", AT=[0,0,2], RELATIVE="Source")
guide.set_comment = "Beam extraction and first guide piece"

In [10]:
guide.show_parameters() # Lets view the parameters available in our guide component

 ___ Help Guide_gravity _____________________________________________________________
|optional parameter|required parameter|default value|user specified value|
w1 [m] // Width at the guide entry
h1 [m] // Height at the guide entry
w2 = 0.0 [m] // Width at the guide exit. If 0, use w1.
h2 = 0.0 [m] // Height at the guide exit. If 0, use h1.
l [m] // length of guide
R0 = 0.995 [1] // Low-angle reflectivity
Qc = 0.0218 [AA-1] // Critical scattering vector
alpha = 4.38 [AA] // Slope of reflectivity
m = 1.0 [1] // m-value of material. Zero means completely absorbing. m=0.65  
               glass/SiO2 Si   Ni  Ni58  supermirror Be    Diamond m=  0.65       
               0.47 1   1.18  2-6         1.01  1.12 for glass/SiO2, m=1 for Ni, 
               1.2 for Ni58, m=2-6 for supermirror. m=0.47 for Si 
W = 0.003 [AA-1] // Width of supermirror cut-off
nslit = 1.0 [1] // Number of vertical channels in the guide (>= 1) (nslit-1 
                   vertical dividing walls). 
d = 0.0005 [m] //

In [11]:
guide.set_parameters(w1=0.05, w2=0.05, h1=0.05, h2=0.05, l=8, m=3.5, G=-9.2)

In [12]:
print(guide) # Verify the information on this component is correct

COMPONENT Guide = Guide_gravity
  w1 = 0.05 [m]
  h1 = 0.05 [m]
  w2 = 0.05 [m]
  h2 = 0.05 [m]
  l = 8 [m]
  m = 3.5 [1]
  G = -9.2 [m/s2]
AT [0, 0, 2] RELATIVE Source


Keyword arguments can be used to specify properties like location when a component is added.

In [13]:
# Add a sample to the instrument
sample = instr.add_component("sample", "PowderN", AT=[0, 0, 9], RELATIVE="Guide") 

In [14]:
# Set parameters corresponding to a copper cylinder
sample.radius = 0.015
sample.yheight = 0.05
sample.reflections = '"Cu.laz"'

In [15]:
instr.available_components("monitors") # Monitors are needed to record information

Here are all components in the monitors category.
 Brilliance_monitor      Monitor_Sqw           Pol_monitor
 Cyl_monitor             Monitor_nD            PreMonitor_nD
 DivLambda_monitor       PSD_TOF_monitor       Res_monitor
 DivPos_monitor          PSD_monitor           Sqq_w_monitor
 Divergence_monitor      PSD_monitor_4PI       Sqw_monitor
 EPSD_monitor            PSD_monitor_TOF       TOF2E_monitor
 E_monitor               PSD_monitor_psf       TOF2Q_cylPSD_monitor
 Event_monitor_simple    PSD_monitor_psf_eff   TOFLambda_monitor
 Hdiv_monitor            PSDcyl_monitor        TOF_PSD_monitor_rad
 L_monitor               PSDlin_diff_monitor   TOF_cylPSD_monitor
 MeanPolLambda_monitor   PSDlin_monitor        TOF_monitor
 Monitor                 PolLambda_monitor     TOFlog_monitor
 Monitor_4PI             PolTOF_monitor        


In [16]:
# Add 4PI detector to detect all neutrons
sphere = instr.add_component("PSD_4PI", "PSD_monitor_4PI", RELATIVE="sample")

In [17]:
sphere.nx = 300
sphere.ny = 300
sphere.radius = 1
sphere.restore_neutron = 1
sphere.filename = '"PSD_4PI.dat"' # filenames need printed quotes, use both ' and "
print(sphere) # Verify that monitors have filenames that are strings when printed

COMPONENT PSD_4PI = PSD_monitor_4PI
  nx = 300 [1]
  ny = 300 [1]
  filename = "PSD_4PI.dat" [string]
  radius = 1 [m]
  restore_neutron = 1 [1]
AT [0, 0, 0] RELATIVE sample


In [18]:
# Add PSD monitor to see the direct beam after the sample
PSD = instr.add_component("PSD", "PSD_monitor", AT=[0,0,1], RELATIVE="sample") 
PSD.xwidth = 0.1
PSD.yheight = 0.1
PSD.nx = 200
PSD.ny = 200
PSD.filename = '"PSD.dat"'
PSD.restore_neutron = 1

In [19]:
L_mon = instr.add_component("L_mon", "L_monitor", RELATIVE="PSD")

In [20]:
# Since the wavelength is an instrument parameter, it can be used when setting parameters
L_mon.Lmin = "wavelength - 0.1"
L_mon.Lmax = "wavelength + 0.1"; L_mon.nL = 150
L_mon.xwidth = 0.1
L_mon.yheight = 0.1
L_mon.filename = '"wave.dat"'
L_mon.restore_neutron = 1
L_mon.comment = "Wavelength monitor for narrow range"

In [21]:
print(L_mon)

// Wavelength monitor for narrow range
COMPONENT L_mon = L_monitor
  nL = 150 [1]
  filename = "wave.dat" [string]
  xwidth = 0.1 [m]
  yheight = 0.1 [m]
  Lmin = wavelength - 0.1 [AA]
  Lmax = wavelength + 0.1 [AA]
  restore_neutron = 1 [1]
AT [0, 0, 0] RELATIVE PSD


In [22]:
instr.show_components() # Lets get an overview of the instrument so far

Source  Source_simple   AT      (0, 0, 0) ABSOLUTE       
Guide   Guide_gravity   AT      (0, 0, 2) RELATIVE Source
sample  PowderN         AT      (0, 0, 9) RELATIVE Guide 
PSD_4PI PSD_monitor_4PI AT      (0, 0, 0) RELATIVE sample
PSD     PSD_monitor     AT      (0, 0, 1) RELATIVE sample
L_mon   L_monitor       AT      (0, 0, 0) RELATIVE PSD   


In [23]:
instr.show_parameters()

double wavelength  = 3  // [AA] Wavelength emmited from source


# Running the McStas instrument
Now we have assembled an instrument and it is time to perform a simulation

In [24]:
# output_path specifies the foldername, if it already exists an index is added
instr.settings(output_path="jupyter_demo", mpi=4, ncount=2E7, gravity=True)
instr.show_settings() # Check settings are correct

Instrument settings:
  ncount:           2.00e+07
  mpi:              4
  gravity:          True
  output_path:      jupyter_demo
  run_path:         .
  package_path:     /Applications/McStas-2.7.1.app/Contents/Resources/mcstas/2.7.1
  executable_path:  /Applications/McStas-2.7.1.app/Contents/Resources/mcstas/2.7.1/bin/
  executable:       mcrun
  force_compile:    True


In [25]:
# Input parameters are set with set_parameters
instr.set_parameters(wavelength=1.5)
instr.show_parameters()

double wavelength  = 1.5  // [AA] Wavelength emmited from source


In [26]:
# The simulation is performed by calling backengine()
data = instr.backengine()

INFO: Using directory: "/Users/madsbertelsen/PaNOSC/McStasScript/github/McStasScript/examples/jupyter_demo"
INFO: Regenerating c-file: jupyter_demo.c
CFLAGS=
INFO: Recompiling: ./jupyter_demo.out
mccode-r.c:1880:1: warning: non-void function does not return a value in all control paths [-Wreturn-type]
} /* mcsiminfo_init */
^
mccode-r.c:2837:3: warning: expression result unused [-Wunused-value]
  *t0;
  ^~~
2 warnings generated.
INFO: ===
         (negative time, miss next components, rounding errors, Nan, Inf).
         (negative time, miss next components, rounding errors, Nan, Inf).
         (negative time, miss next components, rounding errors, Nan, Inf).
         (negative time, miss next components, rounding errors, Nan, Inf).
INFO: Placing instr file copy jupyter_demo.instr in dataset /Users/madsbertelsen/PaNOSC/McStasScript/github/McStasScript/examples/jupyter_demo

Simulation 'jupyter_demo' (jupyter_demo.instr): running on 4 nodes (master is 'CI0021617', MPI version 3.1).
Open

## Working with the returned data
The returned data object is a list of McStasData objects, each containing the results from a monitor.
These data objects also contain preferences for how they should be plotted if this is done automatically.

In [28]:
wavelength_data = ms.name_search("L_mon", data)
wavelength_intensity = wavelength_data.Intensity
wavelength_xaxis = wavelength_data.xaxis

for index in range(70,75):
    print([wavelength_xaxis[index], wavelength_intensity[index]])

[1.494, 475.1862504]
[1.495333333, 474.9090911]
[1.496666667, 473.1532749]
[1.498, 472.6454829]
[1.499333333, 475.884394]


## Plotting the returned data
The plot options looks at some metadata in the McStasData for plotting preferences. For this reason these options can be adjusted for individual data files instead of complex syntax for the plotting command.

In [ ]:
# Adjusting PSD_4PI plot
ms.name_plot_options("PSD_4PI", data, log=1, colormap="hot", orders_of_mag=5)

plot = ms.make_sub_plot(data) # Making subplot of our monitors